# pyESM WORKFLOW

CASE A 

Generation of a new model from scratch, including database and data input

(settings -> use_existing_database: False)

In [1]:
# Create an instance of Model class based on the model settings.
# This generates the model's set input file/s to be filled 
# and the blank SQL database.
from src.frontend.model import Model

test = Model(
    log_level='debug',
    log_format='minimal',
)

INFO | Model | 'Model' object initialization...
INFO | Model.file_manager | 'FileManager' object generated.
INFO | Model | 'Model' object: loading settings.
DEBUG | Model.file_manager | File 'settings.yml' loaded.
INFO | Model | 'Model' object: loading paths from settings.
INFO | Model.core | 'Core' object initialization...
WARNING | Model.file_manager | Directory 'test_case_1' already exists.
DEBUG | Model.file_manager | Directory 'test_case_1' overwritten.
INFO | Model.core.sql_manager | 'SQLManager' object Generated.
INFO | Model.core.index | 'Index' object initialization...
DEBUG | Model.core.index | Loading variables table headers and label headers to Index.
INFO | Model.core.index | 'Index' object initialized.
INFO | Model.core.database | 'Database' object initialization...
DEBUG | Model.core.sql_manager | Connection to 'database.db' opened.
DEBUG | Model.core.sql_manager | SQLite table '_set_SCENARIOS' - created.
DEBUG | Model.core.sql_manager | SQLite table '_set_SCENARIOS' - e

In [2]:
# load model sets from the xlsx files filled by the user
# Note: data validation in excel according to the constant file is envisaged
test.load_model_sets()

INFO | Model.core.index | 'Index' object: loading new Sets to Index.
DEBUG | Model.file_manager | Excel file 'sets.xlsx' loaded.
DEBUG | Model.core.sql_manager | Connection to 'database.db' opened.
INFO | Model.core.database | 'Database' object: loading Sets.
DEBUG | Model.core.sql_manager | SQLite table '_set_SCENARIOS' - 1 entries added.
DEBUG | Model.core.sql_manager | SQLite table '_set_DATETIME' - 2 entries added.
DEBUG | Model.core.sql_manager | SQLite table '_set_TECHNOLOGIES' - 4 entries added.
DEBUG | Model.core.sql_manager | SQLite table '_set_FLOWS' - 4 entries added.
DEBUG | Model.core.sql_manager | Connection to 'database.db' closed.


In [4]:
# Once sets has compiled in excel and loaded in the database, 
# blank database is generated.
# Note: an error is raised at the first launch of the method, 
# disappearing the second run.
test.generate_blank_database()

DEBUG | Model.core.sql_manager | Connection to 'database.db' opened.
INFO | Model.core.database | Generation of empty SQLite database.
DEBUG | Model.core.index | Loading variables coordinates to Index.
DEBUG | Model.core.index | Loading foreign keys to Index.
DEBUG | Model.core.sql_manager | SQLite table 'v' already exists.
DEBUG | Model.core.sql_manager | SQLite 'v' - deleted.
DEBUG | Model.core.sql_manager | SQLite table 'v' - created with foreign keys.
DEBUG | Model.core.sql_manager | SQLite table 'v' - 18 entries added.
DEBUG | Model.core.sql_manager | Column 'values' added to table 'v'.
DEBUG | Model.core.sql_manager | SQLite table 'u' - created with foreign keys.
DEBUG | Model.core.sql_manager | SQLite table 'u' - 12 entries added.
DEBUG | Model.core.sql_manager | Column 'values' added to table 'u'.
DEBUG | Model.core.sql_manager | SQLite table 'Y' - created with foreign keys.
DEBUG | Model.core.sql_manager | SQLite table 'Y' - 4 entries added.
DEBUG | Model.core.sql_manager | Co

In [5]:
# Load input file/s data into the sql database.
# Can be used to update data in the sql database without generating a new 
# Model instance
test.load_data_files_to_database()

DEBUG | Model.core.sql_manager | Connection to 'database.db' opened.
INFO | Model.core.database | Loading data input file/s filled by the user to SQLite database.
DEBUG | Model.file_manager | Excel file 'input_data.xlsx' loaded.
DEBUG | Model.core.sql_manager | 18 rows deleted from table 'v'
DEBUG | Model.core.sql_manager | SQLite table 'v' - 18 entries added.
DEBUG | Model.core.sql_manager | 12 rows deleted from table 'u'
DEBUG | Model.core.sql_manager | SQLite table 'u' - 12 entries added.
DEBUG | Model.core.sql_manager | 4 rows deleted from table 'Y'
DEBUG | Model.core.sql_manager | SQLite table 'Y' - 4 entries added.
DEBUG | Model.core.sql_manager | Connection to 'database.db' closed.


CASE B 

Generation of a new model working with existing database and data input files.

(settings -> use_existing_database: True)

In [ ]:
# Generating a Model instance referring to existing model folder and input data.
from src.frontend.model import Model

test = Model(
    log_level='debug',
    log_format='minimal',
)

test.load_model_sets()

In [ ]:
# In case it is needed, update data without re-generate a new instance of Model class.
test.load_data_files_to_database()

TESTS ---

In [6]:
test.core.index.sets.technologies

symbol: t
table_name: _set_TECHNOLOGIES
table_headers: {'id': ['t_ID', 'INTEGER PRIMARY KEY'], 'name': ['t_Name', 'TEXT'], 'category': ['t_Category', 'TEXT']}
set_categories: {'t.s': 'Supply technology', 't.d': 'Demand technology'}
values:    t_ID           t_Name         t_Category
0     0    Steel factory  Supply technology
1     1  Gas power plant  Supply technology
2     2   PV power plant  Supply technology
3     3     Final demand  Demand technology

In [11]:
test.core.index.variables.u

symbol: u
name: use coefficients matrix
type: exogenous
set_headers_key: name
coordinates_info: {'scenarios': None, 'datetime': None, 'flows': {'set_categories': 'f.p', 'aggregation_key': 'competition'}, 'technologies': {'set_categories': 't.s'}}
var_dict_hierarchy: None
shape: ['flows', 'technologies']
variable_fields: {'scenarios': ['s_Name', 'TEXT'], 'datetime': ['dt_Name', 'TEXT'], 'flows': ['f_Name', 'TEXT'], 'technologies': ['t_Name', 'TEXT']}
table_headers: {'id': ['id', 'INTEGER PRIMARY KEY'], 'scenarios': ['s_Name', 'TEXT'], 'datetime': ['dt_Name', 'TEXT'], 'flows': ['f_Name', 'TEXT'], 'technologies': ['t_Name', 'TEXT']}
coordinates: {'scenarios': ['Reference'], 'datetime': [2023, 2024], 'flows': ['Energy', 'Steel'], 'technologies': ['Steel factory', 'PV power plant', 'Gas power plant']}
foreign_keys: {'s_Name': ('s_Name', '_set_SCENARIOS'), 'dt_Name': ('dt_Name', '_set_DATETIME'), 'f_Name': ('f_Name', '_set_FLOWS'), 't_Name': ('t_Name', '_set_TECHNOLOGIES')}

In [ ]:
test.model.database.sqltools.open_connection()

var = test.model.database.sqltools.filtered_table_to_dataframe(
    table_name='v',
    filters_dict={
        's_Name': ['Reference'],
        'dt_Name': [2023],
        't_Name': ['Gas power plant'],
        'f_Name': ['Steel']
    }
)

filter_1 = test.model.database.sqltools.get_related_table_keys(
    parent_table_name='_set_FLOWS',
    child_column_name='f_Name',
    parent_table_fields={
        'f_Category': ['Product flow'],
    }
)

filter_2 = test.model.database.sqltools.get_related_table_keys(
    parent_table_name='_set_DATETIME',
    child_column_name='dt_Name',
    parent_table_fields={
        'dt_Name': [2023],
    }
)

var_custom_filter = test.model.database.sqltools.filtered_table_to_dataframe(
    table_name='v',
    filters_dict={
        **filter_1,
        **filter_2,
        **{'f_Name': ['Steel']},
    }
)

test.model.database.sqltools.close_connection()

# filter_1, filter_2
var_custom_filter

In [ ]:
# delete all the information included in the model data folder, if necessary
test.core.model_dir_cleanup()